In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import os
import numpy as np
os.chdir("C:/Users/mihir/OneDrive/Desktop/MDI/Gurobi_Workspace/Cracking_the_vault_2025_GDDA")

In [7]:
knapsacks = pd.read_csv("knapsacks.csv", index_col=0).squeeze("columns")
items = pd.read_csv("items.csv", index_col=0).squeeze("columns")

weight_series = pd.Series(items.Weight.tolist(), index=items.index.tolist(), name = "weight")
price_series = pd.Series(items.Price.tolist(), index=items.index.tolist(), name = "price")
duplicates_series = pd.Series(items.Max_Duplicates.tolist(), index=items.index.tolist(), name = "duplicates")
knapsacks_series = pd.Series(knapsacks.tolist(), index=knapsacks.index.tolist(), name = "knapsacks")


In [23]:
heist_model = gp.Model("heist")

In [25]:
is_assigned = heist_model.addVars(knapsacks_series.index, items.index, name="is_assigned", vtype=GRB.INTEGER)
heist_model.update()

In [26]:
duplicates_constr = heist_model.addConstrs(((gp.quicksum(is_assigned[k,i] for k in knapsacks_series.index))<=duplicates_series[i] for i in duplicates_series.index))
heist_model.update()

In [27]:
weight_constr = heist_model.addConstrs(((gp.quicksum(is_assigned[k,i]*weight_series[i] for i in weight_series.index))<=knapsacks_series[k] for k in knapsacks_series.index))
heist_model.update()

In [30]:
heist_model.setObjective(gp.quicksum(price_series[i] * gp.quicksum(is_assigned[k,i] for k in knapsacks_series.index)
    for i in price_series.index), GRB.MAXIMIZE)
heist_model.update()

In [31]:
heist_model.write("heist.lp")
heist_model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 70 rows, 1000 columns and 2000 nonzeros
Model fingerprint: 0xce0dfb73
Variable types: 0 continuous, 1000 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 377.8000000
Presolve removed 0 rows and 210 columns
Presolve time: 0.03s
Presolved: 70 rows, 790 columns, 1580 nonzeros
Variable types: 0 continuous, 790 integer (495 binary)

Root relaxation: objective 6.674000e+02, 177 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [52]:
sol = pd.Series(heist_model.getAttr('X', is_assigned), name = 'selections')
sol_frame = sol.to_frame()
display(heist_model.ObjVal)
sol_frame[sol_frame["selections"]>0]

667.4

selections
Knapsack_1  Item_4          1.0
            Item_22         2.0
Knapsack_2  Item_14         2.0
            Item_26         3.0
            Item_46         1.0
Knapsack_3  Item_20         1.0
            Item_33         2.0
Knapsack_4  Item_20         3.0
            Item_28         3.0
Knapsack_5  Item_4          1.0
            Item_10         1.0
Knapsack_6  Item_21         2.0
            Item_39         1.0
Knapsack_7  Item_11         1.0
            Item_26         1.0
            Item_28         1.0
            Item_46         1.0
Knapsack_8  Item_21         1.0
            Item_34         1.0
Knapsack_9  Item_33         1.0
Knapsack_10 Item_34         1.0
            Item_47         1.0
Knapsack_11 Item_2          2.0
            Item_7          2.0
Knapsack_12 Item_10         1.0
            Item_14         1.0
Knapsack_13 Item_2          1.0
            Item_28         1.0
            Item_40         1.0
Knapsack_14 Item_10         1.0
            Item_22         1.0
Knapsack_15 Item_2          1.0
            Item_14         1.0
Knapsack_16 Item_2          1.0
            Item_26         1.0
Knapsack_17 Item_4          1.0
            Item_47         2.0
Knapsack_18 Item_20         1.0
            Item_34         1.0
Knapsack_19 Item_32         4.0
Knapsack_20 Item_33         1.0